In [1]:
#%pip install --upgrade langchain langsmith langgraph langchain_openai openai python-dotenv javascript
# run this in terminal: python3 -m javascript --install @near-lake/primitives
%reload_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv('.env')

# Set model variables
OPENAI_BASE_URL = "https://api.openai.com/v1"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION")


os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "Indexer Agent Local"


# Execute the Agent

In [3]:
from langchain_core.messages import HumanMessage
from graph.trivial_agent_graph import IndexerAgentGraphBuilder
from agents.IndexerAgent import indexer_agent_model, tool_executor
from tools.JavaScriptRunner import tool_js_on_block, tool_js_on_block_schema
import json

inputs = {
    "messages": [
        HumanMessage(
            content="Write JavaScript code for receiver app.nearcrowd.near that returns all function calls with their arguments converted from base64, block number, date and receiptId",
        ),
        HumanMessage(
            content="You can use various block heights like 119688211, 119688186, 119688185 to understand the structure of the blocks by running JavaScript code on them.",
        )
    ]
}

# Define the tools that the agent will use
tools = [tool_js_on_block, tool_js_on_block_schema]
model = indexer_agent_model(tools)

# print(model.invoke(input=inputs['messages']))

builder = IndexerAgentGraphBuilder(model, tool_executor(tools))
app = builder.graph()

result = app.with_config({"run_name": "IndexerAgentPavel"}).invoke(inputs)

# for output in app.with_config({"run_name": "IndexerAgentPavel"}).stream(inputs):
#     # stream() yields dictionaries with output keyed by node name
#     for key, value in output.items():
#         print(f"----------------- Output from node '{key}':")
#         print(json.dumps(value, indent=2))
#     print("\n---\n")


RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for gpt-4 in organization org-dPyrYIP9hbS1vIq5YkNAQ5F5 on tokens per min (TPM): Limit 300000, Requested 920377. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:
result['messages'][-1]

In [3]:

from tools.JavaScriptRunner import run_js_on_block_only_schema
code = """
return block.actions()
    .filter(a => a.receiverId==='app.nearcrowd.near')
    .flatMap(a => a.operations.filter(op => !!op.FunctionCall))
    .map(op => {
        const functionCall = op.FunctionCall;
        const args = Buffer.from(functionCall.args, 'base64').toString();
        return {
            blockNumber: block.blockHeight,
            date: new Date(parseInt(block.header().timestampNanosec) / 1000000),
            methodName: functionCall.methodName,
            args: JSON.parse(args),
            gas: functionCall.gas,
            deposit: functionCall.deposit
        };
    });
"""
print(run_js_on_block(119688211, code.replace("\\n", "")))

TypeError: the JSON object must be str, bytes or bytearray, not list